GAN Variants -
1. DC GAN (Deep Convolutional GAN)
2. Conditional GAN 
3. Cycle GAN 

DC GAN : Its a type of GANs that specifically designed for generating realistic images by using deep convolutional neural networks intead of fully connected layer.It was introduced in 2015 .

How DC GAN Works :
Generator -
1. Random Inputs.
2. Transpose Convolutional layer 
3. Batch Normalization 
4. Relu

Discriminator -
1. Take generated image as a input
2. Use Convolutional layer (Conv2D)
3. Leaky Relu 
4. No pooling layer 

Real World Applications :
1. Fashion Related 
2. Face Generation 
3. Data Augumentation



In [1]:
#DC GAN 
import tensorflow as tf 
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import os 
from tensorflow.keras import layers 

In [2]:
#hyper parameters 
buffer_size = 6000
batch_size = 255 
image_height = 28 
image_width = 28
Channels = 1 
Epochs = 30
Latent_dim =100

In [3]:
#Load and Preprocess Datasets
def load_dataset():
    (x_train , y_train),(x_test , y_test ) = tf.keras.datasets.fashion_mnist.load_data()
    x_train = x_train.reshape((-1,image_height , image_width , Channels)).astype("float32")
    x_train = (x_train - 127.5)/127.5
    return x_train

In [4]:
x_train = load_dataset()
train_dataset = tf.data.Dataset.from_tensor_slices(x_train).shuffle(buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [5]:
#Build Generators --
def build_generators():
    model=tf.keras.Sequential(name="Generator")
    model.add(layers.Dense(7*7*255,use_bias=False , input_shape = (Latent_dim,)))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    model.add(layers.Reshape((7,7,255)))
    #Up Sample 14*14
    model.add(layers.Conv2DTranspose(128,kernel_size=5,strides=2 ,padding="same",use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    #28*28
    model.add(layers.Conv2DTranspose(128,kernel_size=5,strides=2,padding="same",use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    #Final Layers --
    model.add(layers.Conv2DTranspose(Channels,kernel_size=5,strides=1 ,padding="same",use_bias=False,activation="tanh"))
    return model 

In [6]:
generators = build_generators()
generators.summary()

c:\Users\hp\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\core\dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "Generator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 12495)          │     1,249,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 12495)          │        49,980 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu (ReLU)                    │ (None, 12495)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reshape (Reshape)               │ (None, 7, 7, 255)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose                │ (None, 14, 14, 128)    │       816,000 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 14, 14, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_1 (ReLU)                  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_1              │ (None, 28, 28, 128)    │       409,600 │
│ (Conv2DTranspose)               │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 28, 28, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ re_lu_2 (ReLU)                  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_transpose_2              │ (None, 28, 28, 1)      │         3,200 │
│ (Conv2DTranspose)               │                        │               │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,529,304 (9.65 MB)

 Trainable params: 2,503,802 (9.55 MB)

 Non-trainable params: 25,502 (99.62 KB)

In [7]:
#Build Discrimainators --
def build_discriminators():
    model=tf.keras.Sequential(name="Discriminator")
    model.add(layers.Conv2D(64,kernel_size=5,strides=2 ,padding="same",use_bias=False , input_shape = [image_height,image_width,Channels]))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))
    model.add(layers.Conv2D(128,kernel_size=5,strides=2,padding="same",use_bias=False))
    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    return model

In [8]:
discriminator = build_discriminators()
discriminator.summary()

c:\Users\hp\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\hp\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\activations\leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "Discriminator"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 14, 14, 64)     │         1,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ leaky_re_lu (LeakyReLU)         │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 7, 7, 128)      │       204,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │         6,273 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 212,673 (830.75 KB)

 Trainable params: 212,673 (830.75 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
#Losses and Optimizers --
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
def loss_discriminators(real,fake):
    real_loss = cross_entropy(tf.ones_like(real),real)
    fake_loss = cross_entropy(tf.zeros_like(fake),fake)
    return real_loss + fake_loss 
def loss_generators(fake):
    return cross_entropy(tf.ones_like(fake),fake)

In [10]:
#Optimizers --
generator_optimizers = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizers = tf.keras.optimizers.Adam(1e-4)

In [11]:
#
Number_of_sample = 15
output_dir = "Generated_Image"
checkpoint_dir = "Checkpoints"

os.makedirs(output_dir, exist_ok=True)
os.makedirs(checkpoint_dir, exist_ok=True)  

In [13]:
#Input for consistent inputs --
inputs = tf.random.normal([Number_of_sample,Latent_dim])
print(inputs)

tf.Tensor(
[[ 0.5922262  -0.5258798  -0.8433983  ... -1.8937172   1.5644685
  -0.02482687]
 [-0.20231105  1.0216639   1.2551847  ...  0.6893802  -1.9591879
  -0.7570724 ]
 [ 0.10143691  1.4568998   1.7129158  ...  0.0422195   0.38731384
   0.60968274]
 ...
 [-1.7031933  -1.147244    0.08040026 ... -1.514592   -0.01218207
   0.50807446]
 [-0.05449166  0.7690648  -0.12393246 ... -0.8742377  -0.02722833
  -1.0019344 ]
 [-0.1647347   0.9896195   0.9642268  ...  0.21821225 -0.851112
   0.39351833]], shape=(15, 100), dtype=float32)


In [14]:
#Check Point-----
Checkpoint1 = os.path.join(checkpoint_dir,"ckpt")

In [ ]:
Checkpoint2 = tf.train.Checkpoint(generator_optimizers=generator_optimizers,
                                  discriminator_optimizers=discriminator_optimizers,
                                  generators=generators,
                                  discriminator=discriminator)

In [21]:
#Checkpoint Manager --
Checkpoint_manager = tf.train.CheckpointManager(Checkpoint2,checkpoint_dir,max_to_keep=5)

In [25]:
#Restore the latest checkpoint ---
if Checkpoint_manager.latest_checkpoint:
    print("Restoring from Checkpoint2:",Checkpoint_manager.latest_checkpoint)
    Checkpoint2.restore(Checkpoint_manager.latest_checkpoint)

In [52]:
#Generate and Save Images --
def generate_and_save_images(model, epochs, test_input,grid_size=(4,4)):
    predictions = model(test_input, training=False)
    predict = (predictions + 1) / 2.0  # Rescale to [0, 1]
    prediction1 = predict.numpy()
    fig = plt.figure(figsize=(grid_size[1]*2, grid_size[0]*2))
    for i in range(prediction1.shape[0]):
        plt.subplot(grid_size[0], grid_size[1], i + 1)
        img = prediction1[i, :, :, 0]
        plt.imshow(img, cmap='gray')
    plt.title(f"epochs{Epochs}")
    file_name = os.path.join(output_dir, f"{epochs}.png")
    plt.savefig(file_name,bbox_inches='tight') 
    plt.close(fig)       
   

In [53]:
generate_and_save_images(generators, Epochs, inputs)        

In [ ]:
#Training Loop--
@tf.function
def train(images):
    input  = tf.random.normal([batch_size, Latent_dim])
    with tf.GradientTape() as disc_tape, tf.GradientTape() as gen_tape:
        generated_images = generators(input, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        disc_loss = loss_discriminators(real_output, fake_output)
        gen_loss = loss_generators(fake_output)
    gra_of_dis = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    gra_of_gen = gen_tape.gradient(gen_loss, generators.trainable_variables)
    discriminator_optimizers.apply_gradients(zip(gra_of_dis, discriminator.trainable_variables))
    generator_optimizers.apply_gradients(zip(gra_of_gen, generators.trainable_variables))
    return disc_loss, gen_loss

In [ ]:
def train1(dataset, epochs):
    st = time.time()
    for epoch in range(1,epochs+1):
        epochs_gen_loss